# Orchestrator Notebook
This notebook runs all the desired pipeline processes in sequence.



## Project root setup

This section ensures the notebook runs modules with `python -m src...` by setting CWD and sys.path.


In [1]:
import os, sys, pathlib

def _resolve_project_root(start: pathlib.Path, marker: pathlib.Path) -> pathlib.Path:
    """Walk upwards from `start` until a directory containing `marker` is found."""
    for candidate in (start, *start.parents):
        if (candidate / marker).exists():
            return candidate
    return start

START_DIR = pathlib.Path(os.getcwd()).resolve()
NOTEBOOK_MARKER = pathlib.Path("src")
ROOT = _resolve_project_root(START_DIR, NOTEBOOK_MARKER)

if START_DIR != ROOT:
    os.chdir(ROOT)

# Add the project root to sys.path (so 'import src.*' works)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("Notebook start dir:", START_DIR)
print("CWD:", os.getcwd())
print("sys.executable:", sys.executable)


Notebook start dir: D:\cosas_uni\ADSDB-Project\src
CWD: D:\cosas_uni\ADSDB-Project
sys.executable: C:\Users\pablo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


## Configuration and modules definition

Defines each module in order to be executed.

In [2]:
# --- Configuration ---
from pathlib import Path
import os


MODULES = [
    # --------- Managers ---------
    {
        "name": "MinIO manager",
        "module": "src.common.minio_manager",  
        "args": []
    },
    {
        "name": "ChromaDB manager",
        "module": "src.common.chroma_client", 
        "args": []
    },

    # --------- Data ingestion ---------
    {
        "name": "Text ingestion",
        "module": "src.data_management.data_ingestion.text_ingestion",
        "args": []
    },
    {
        "name": "Image ingestion",
        "module": "src.data_management.data_ingestion.image_ingestion",
        "args": []
    },
    {
        "name": "Video ingestion",
        "module": "src.data_management.data_ingestion.video_ingestion",
        "args": []
    },

    # --------- Landing zone --------- 
    {
        "name": "Landing zone",
        "module": "src.data_management.landing_zone.landing_zone",
        "args": []
    },


    # --------- Formatted zone --------- 
    {
        "name": "Formatted text",
        "module": "src.data_management.formatted_zone.formatted_text",
        "args": []
    },
    {
        "name": "Formatted images",
        "module": "src.data_management.formatted_zone.formatted_images",
        "args": []
    },
    {
        "name": "Formatted videos",
        "module": "src.data_management.formatted_zone.formatted_videos",
        "args": []
    },


    # --------- Trusted zone --------- 
    {
        "name": "Trusted text",
        "module": "src.data_management.trusted_zone.trusted_text",
        "args": []
    },
    {
        "name": "Trusted images",
        "module": "src.data_management.trusted_zone.trusted_images",
        "args": []
    },
    {
        "name": "Trusted videos",
        "module": "src.data_management.trusted_zone.trusted_videos",
        "args": []
    }, 

    # --------- Exploitation zone ---------
    {
        "name": "Exploitation text",
        "module": "src.data_management.exploitation_zone.exploitation_text",
        "args": []
    },
    {
        "name": "Exploitation images",
        "module": "src.data_management.exploitation_zone.exploitation_images",
        "args": []
    },
    {
        "name": "Exploitation videos",
        "module": "src.data_management.exploitation_zone.exploitation_videos",
        "args": []
    },

    # --------- Tasks ---------
    {
        "name": "Same-Modality Task",
        "module": "src.multi_modal_tasks.same_modality_task",
        "args": []
    },
    {
        "name": "Multi-Modality Task",
        "module": "src.multi_modal_tasks.multi_modality_task",
        "args": []
    },
    {
        "name": "Generative Task",
        "module": "src.multi_modal_tasks.generative_task",
        "args": []
    }
]


## Module selection
Use variable **<i>RUN_ONLY</i>** to whitelist the pipeline steps you **actually want to execute**—list their names or zero-based indexes and any unspecified steps will be skipped—while variable **<i>SKIP_STEPS</i>** provides the opposite filter, letting you **exclude steps** by name or index even if they would otherwise run; if both filters remove every module the notebook raises an error, so make sure at least one module slips through. If both variables are empty, the whole pipeline will execute. If you want to use integers, here is the list of processes:

0. MinIO manager
1. ChromaDB manager
2. Text ingestion
3. Image ingestion
4. Video ingestion
5. Landing zone
6. Formatted text
7. Formatted images
8. Formatted videos
9. Trusted text
10. Trusted images
11. Trusted videos
12. Exploitation text
13. Exploitation images
14. Exploitation videos
15. Same-Modality Task
16. Multi-Modality Task
17. Generative Task

In [3]:
from typing import Iterable, Sequence, Union, Tuple, Set

# Configure which modules should run.
# Leave RUN_ONLY and SKIP_STEPS empty to execute every module.
RUN_ONLY: Sequence[Union[str, int]] = [6, 7, 9, 10, 12, 13]  # e.g. ['Image ingestion', 3]
SKIP_STEPS: Sequence[Union[str, int]] = []  # e.g. ['Formatted videos', 5]

def _divide_targets(targets: Iterable[Union[str, int]]) -> Tuple[Set[str], Set[int]]:
    names: Set[str] = set()
    indexes: Set[int] = set()
    for target in targets:
        if isinstance(target, str):
            names.add(target.strip())
        elif isinstance(target, int):
            indexes.add(target)
        elif target is None:
            continue
        else:
            raise TypeError(f'Unsupported selector type: {type(target)!r}')
    return names, indexes

def _should_include(idx: int, name: str, only_names: Set[str], only_idx: Set[int], skip_names: Set[str], skip_idx: Set[int]) -> bool:
    if only_names or only_idx:
        if name not in only_names and idx not in only_idx:
            return False
    if name in skip_names or idx in skip_idx:
        return False
    return True

only_names, only_idx = _divide_targets(RUN_ONLY)
skip_names, skip_idx = _divide_targets(SKIP_STEPS)

MODULES_TO_RUN = [
    step for idx, step in enumerate(MODULES)
    if _should_include(idx, step['name'], only_names, only_idx, skip_names, skip_idx)
]

if not MODULES_TO_RUN:
    raise ValueError('The actual configuration does not run any module. Please set RUN_ONLY and SKIP_STEPS correctly.')

print("Modules available:")
for idx, step in enumerate(MODULES):
    marker = 'RUN ' if step in MODULES_TO_RUN else 'SKIP'
    print(f"  [{idx:02d}] {marker} {step['name']}")

print("To be executed:", ', '.join(step['name'] for step in MODULES_TO_RUN))


Modules available:
  [00] SKIP MinIO manager
  [01] SKIP ChromaDB manager
  [02] SKIP Text ingestion
  [03] SKIP Image ingestion
  [04] SKIP Video ingestion
  [05] SKIP Landing zone
  [06] RUN  Formatted text
  [07] RUN  Formatted images
  [08] SKIP Formatted videos
  [09] RUN  Trusted text
  [10] RUN  Trusted images
  [11] SKIP Trusted videos
  [12] RUN  Exploitation text
  [13] RUN  Exploitation images
  [14] SKIP Exploitation videos
  [15] SKIP Same-Modality Task
  [16] SKIP Multi-Modality Task
  [17] SKIP Generative Task
To be executed: Formatted text, Formatted images, Trusted text, Trusted images, Exploitation text, Exploitation images


## Run the pipeline

In [4]:
from datetime import datetime
import runpy

print("=== Pipeline start:", datetime.now().isoformat(), "===")

for step in MODULES_TO_RUN:
    print(f"\n=== Step: {step['name']} ===")
    
    # limpiar sys.argv para evitar argumentos del kernel
    sys.argv = [step["module"], *map(str, step.get("args", []))]
    runpy.run_module(step["module"], run_name="__main__")

print("\n=== Pipeline completed successfully ===")

=== Pipeline start: 2025-11-20T21:24:42.028446 ===

=== Step: Formatted text ===


Processing texts:   0%|          | 0/1 [00:00<?, ?file/s]


=== Step: Formatted images ===


Processing images:   0%|          | 0/1500 [00:00<?, ?image/s]


=== Step: Trusted text ===


...


REJECTION REPORT
 - Low contrast: 427 image(s) rejected
 - Bad brightness, Low contrast: 15 image(s) rejected
 - Bad brightness: 2 image(s) rejected

Uploaded to Trusted: 1056
Uploaded to Rejected: 444

 Validated images per food type:
 - Bread: 231
 - Dairy product: 93
 - Dessert: 392
 - Egg: 234
 - Fried food: 106

=== Step: Exploitation text ===
[INFO] Limiting ingestion to the first 500 items.
Processing 500 texts from trusted/text_data/full_format_recipes.json...



Ingesting texts:   0%|          | 0/500 [00:00<?, ?txt/s]


OK - Texts ingested from trusted/text_data/full_format_recipes.json: 500

=== Step: Exploitation images ===
Processing 1056 images from MinIO...



Embedding images:   0%|          | 0/1056 [00:00<?, ?img/s]


OK - Image embeddings ingested into Chroma: 1056

=== Pipeline completed successfully ===
